In [2]:
from pathlib import Path

from gabenet.utils import holdout_split
import haiku as hk
import jax.numpy as jnp
import pandas as pd
from sklearn.model_selection import train_test_split


_PATH = Path('./')
_COSMIC_PATH = _PATH / "COSMIC_v3.3_SBS_GRCh37.txt"
COSMIC_WEIGHTS = pd.read_csv(_COSMIC_PATH, sep="\t", index_col=0).T

In [3]:
def _read_sbs96():
    # Get path of this file and read mutation spectrum.
    df = pd.read_csv(_PATH / "WGS_PCAWG.96.csv")
    df["Type"] = (
        df.Trinucleotide.str[0]
        + "["
        + df["Mutation type"]
        + "]"
        + df.Trinucleotide.str[2]
    )
    # The purpose of this temporary index is for sorting.
    df["sortable_index"] = (
        "[" + df["Mutation type"].str[:3] + "]" + df.Trinucleotide.str[:3]
    )
    df = df.reset_index()
    # Drop sorting index from dataframe.
    df = df.set_index("Type").sort_index()
    df = df.drop(columns=["sortable_index", "index", "Mutation type", "Trinucleotide"])
    df = df.transpose()
    assert jnp.all(df.columns == COSMIC_WEIGHTS.columns)
    return df

In [4]:
df = _read_sbs96()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
df2 = _read_sbs96()

In [7]:
df.append(df2).shape

/tmp/ipykernel_25691/1331713195.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(df2).shape


(5560, 96)